<a href="https://colab.research.google.com/github/girishgupta/Sequence-Models/blob/master/LSTM_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Example script to generate text from Nietzsche's writings.
'''
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io


Using TensorFlow backend.


In [2]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))



606208/600901 [==============================] - 0s 0us/step
corpus length: 600893


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 57
nb sequences: 200285
Vectorization...


In [4]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)




Build model...
Instructions for updating:
Colocations handled automatically by placer.


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_1 (Dense)              (None, 57)                7353      
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
200285/200285 [==============================] - 225s 1ms/step - loss: 1.9842

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " culture of europe, it is still the high"
 culture of europe, it is still the higher that is a presunce of a presument of the more the such any any any any and the pressity and the man in the presunce that any any has any and the presunce of has any any any the presunce that they the any in the presure and the something the explessely in the indelicious and for the more the presist the suncertion that is the consided to the indelight to the presunce of the suncertion of the pre
----- diversity: 0.5
----- Generating with seed: " culture of europe, it is still the high"
 culture of europe, it is still the higher are streng in the
more they have the is the more innecessity and in the enception this the the
appearactical us that as they any of a streath thing that the 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]


all has not be reguse of the spected," our soul, which have
always actures and soul which have always thinkin
----- diversity: 1.0
----- Generating with seed: "lf which
renders obedience, or which he "
lf which
renders obedience, or which he had not be or-giver as as though "part specting, for authorite ones: he
who way of one of, perhaps icmans its
hade decision of the german quige has acporant." i though rapely greatest praised
displook natural fullence can them was be domain of scienct trains--haze and made had it be denopelicisinal to religh is all the criost had to him.for old
of the amppession of bluman cause
with the ordinary m
----- diversity: 1.2
----- Generating with seed: "lf which
renders obedience, or which he "
lf which
renders obedience, or which he cannouse with oranisles
and dressing same men of mace style scholarizested; knowledgned in gaiped,
pannjutheng, sevudctivelest delicarate value of logistane, just an
prosed delogichly
mansation, the "heling and moralsingike d

KeyboardInterrupt: ignored